Read the packages

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from pandas import DataFrame
import math
import numpy
from statistics import *
from numpy import nan

mount to drive

In [ ]:
# Make suitable changes here. My .csv file is in :  My Drive/Colab Notebooks
from google.colab import drive
drive.mount("/content/drive/")
%cd 'national_park_visitation_analysis_w_mobile_phone_data/volume/data/raw/census_files/'
!pwd #Prints the present working directory


Mounted at /content/drive/
/content/drive/My Drive/RTPM496/National Park Research/Income_and_race
/content/drive/My Drive/RTPM496/National Park Research/Income_and_race


Read the dataset

In [ ]:
# Make suitable changes here. My .csv file is in :  My Drive/Colab Notebooks
!pwd #Prints the present working directory
incomedata = pd.read_csv('income.csv')
incomedata.tail()

/content/drive/My Drive/RTPM496/National Park Research/Income_and_race


,Geo_FIPS,total.households,less.than.10000,10000.to.14999,15000.to.19999,20000.to.24999,25000.to.29999,30000.to.34999,35000.to.39999,40000.to.44999,45000.to.49999,50000.to.59999,60000.to.74999,75000.to.99999,100000.to.124999,125000.to.149999,150000.to.199999,200000.or.more,MEDIAN.INCOME
220328,721537506011,237,102,14,45,19,0,0,31,7,19,0,0,0,0,0,0,0,15781.0
220329,721537506012,740,97,140,96,61,97,58,32,28,47,57,8,0,19,0,0,0,23636.0
220330,721537506013,256,48,9,8,0,42,0,0,39,0,12,56,24,18,0,0,0,42955.0
220331,721537506021,560,303,78,91,8,0,11,32,0,0,10,27,0,0,0,0,0,8722.0
220332,721537506022,183,67,25,14,0,51,8,9,0,0,0,0,9,0,0,0,0,NaN


Obtain details of the dataset

In [ ]:
incomedata.shape

(220333, 19)

Extract the data with missing median income information

In [ ]:
missing_income_data=incomedata[incomedata["MEDIAN.INCOME"].isna()]
missing_income_data=missing_income_data[["Geo_FIPS", "total.households", "less.than.10000", "10000.to.14999", "15000.to.19999", "20000.to.24999", "25000.to.29999", "30000.to.34999"
, "35000.to.39999", "40000.to.44999", "45000.to.49999", "50000.to.59999", "60000.to.74999", "75000.to.99999", "100000.to.124999", "125000.to.149999", "150000.to.199999", "200000.or.more"]]
missing_income_data = missing_income_data[missing_income_data['total.households'] != 0]
missing_income_data.head()
median=(missing_income_data["total.households"]+1)/2
flooring=numpy.floor(median)
missing_income_data["median"]=flooring
missing_income_data_temp=missing_income_data.drop(columns=["Geo_FIPS", "total.households", "median"])
the_list=(missing_income_data_temp.values.tolist())
the_cum_list=[]
for i in range(0, len(the_list)):
  a_list=the_list[i]
  x=np.cumsum(a_list)
  the_cum_list.append(list(x))
missing_income_data.tail()


#type(missing_income_data["the_list"])


,Geo_FIPS,total.households,less.than.10000,10000.to.14999,15000.to.19999,20000.to.24999,25000.to.29999,30000.to.34999,35000.to.39999,40000.to.44999,45000.to.49999,50000.to.59999,60000.to.74999,75000.to.99999,100000.to.124999,125000.to.149999,150000.to.199999,200000.or.more,median
220243,721455607013,428,90,136,0,0,54,17,0,29,47,27,0,0,28,0,0,0,214.0
220257,721479505001,258,92,47,0,26,8,34,24,0,18,0,9,0,0,0,0,0,129.0
220302,721519513002,427,205,18,18,15,45,28,0,35,0,19,0,44,0,0,0,0,214.0
220319,721537504004,190,74,39,0,31,0,0,0,0,28,0,8,0,0,10,0,0,95.0
220332,721537506022,183,67,25,14,0,51,8,9,0,0,0,0,9,0,0,0,0,92.0


We have a total of 6720 missing median income in the dataset

We wish to impute the missing data using the following technique below. 

1. Find the median population of the census block group
2. Find which income bracket the median population belongs to
3. We use the center of the income bracket as the median income for the census block group. 

In [ ]:
missing_income_data.shape

(6720, 19)

Step 1: obtain the median population for the census block group with the missing median income data.

In [ ]:
pos=[]
check=[]
flooring_interest=list(flooring)

for i in range(0, len(the_cum_list)):
  #print(i)
  a_list=the_cum_list[i]
  interest=flooring_interest[i]
  #print(a_list)
  for n in range(0, len(a_list)):
    if (a_list[n]) < interest:
      continue
    else:
      #print(a_list)
      pos.append(n)
      check.append(a_list)
      break
len(check)




6720

Step 2: Find which income bracket the median population belongs to

and 

Step 3: Use the center of the income bracket as the median income

In [ ]:
pred_median=[]
for i in range(0, len(pos)):
  x=pos[i]
  if x==1:
    y=9999/2
  if x==2:
    y=(10000+ 14999)/2
  if x==3:
    y=(15000+ 19999)/2
  if x==4:
    y=(20000+ 24999)/2
  if x==5:
    y=(25000+ 29999)/2
  if x==6:
    y=(30000+ 34999)/2
  if x==7:
    y=(35000+ 39999)/2
  if x==8:
    y=(40000+ 44999)/2
  if x==9:
    y=(45000+49999)/2
  if x==10:
    y=(50000+ 59999)/2
  if x==11:
    y=(60000+ 74999)/2
  if x==12:
    y=(75000+ 99999)/2
  if x==13:
    y=(100000+ 124999)/2
  if x==14: 
    y=(125000+ 149999)/2
  if x==15:
    y=(150000+ 199999)/2
  if x==16:
    y=200000
  pred_median.append(y)
len(pred_median)
missing_income_data["MEDIAN.INCOME.PRED"]=pred_median
missing_income_data.head()

,Geo_FIPS,total.households,less.than.10000,10000.to.14999,15000.to.19999,20000.to.24999,25000.to.29999,30000.to.34999,35000.to.39999,40000.to.44999,45000.to.49999,50000.to.59999,60000.to.74999,75000.to.99999,100000.to.124999,125000.to.149999,150000.to.199999,200000.or.more,median,MEDIAN.INCOME.PRED
47,10030106001,411,24,111,80,9,8,13,8,0,0,27,38,73,0,0,0,20,206.0,12499.5
158,10070100041,300,14,32,77,0,19,0,0,14,11,15,19,59,40,0,0,0,150.0,37499.5
164,10090501011,375,0,40,20,54,40,31,18,17,4,9,2,62,39,0,39,0,188.0,32499.5
200,10119522001,520,130,60,36,13,4,0,0,130,0,11,26,0,84,10,16,0,260.0,37499.5
207,10139527002,251,31,20,62,17,0,0,0,0,5,17,55,34,0,0,10,0,126.0,17499.5


Replace the missing data with the imputed data 

In [ ]:


missing_income_data_total = pd.merge(incomedata, missing_income_data, how="left", on=["Geo_FIPS"])
missing_income_data_total.shape

(220333, 38)

In [ ]:
final_list=[]
x=list(incomedata["MEDIAN.INCOME"])
x = [str(l) for l in x]
y=list(missing_income_data_total["MEDIAN.INCOME.PRED"])
y = [str(l) for l in y]

for i in range(0,len(x)):
  income=x[i]
  if income==nan or income=="nan":
    median=y[i]
    final_list.append(median)
  else:
    median=income
    final_list.append(median)

final_list = [float(l) for l in final_list]

In [ ]:
incomedata["MEDIAN.INCOME"]=final_list

write the data. 

In [ ]:
incomedata.tail()

,Geo_FIPS,total.households,less.than.10000,10000.to.14999,15000.to.19999,20000.to.24999,25000.to.29999,30000.to.34999,35000.to.39999,40000.to.44999,45000.to.49999,50000.to.59999,60000.to.74999,75000.to.99999,100000.to.124999,125000.to.149999,150000.to.199999,200000.or.more,MEDIAN.INCOME
220328,721537506011,237,102,14,45,19,0,0,31,7,19,0,0,0,0,0,0,0,15781.0
220329,721537506012,740,97,140,96,61,97,58,32,28,47,57,8,0,19,0,0,0,23636.0
220330,721537506013,256,48,9,8,0,42,0,0,39,0,12,56,24,18,0,0,0,42955.0
220331,721537506021,560,303,78,91,8,0,11,32,0,0,10,27,0,0,0,0,0,8722.0
220332,721537506022,183,67,25,14,0,51,8,9,0,0,0,0,9,0,0,0,0,4999.5


In [ ]:
incomedata.to_csv('income.csv')